In [29]:
import pandas as pd

In [30]:
import numpy as np

In [31]:
import cv2

In [32]:
from datetime import datetime

In [33]:
import os

In [34]:
import face_recognition

In [35]:
# Directories for student images and group photos
individual_images_dir = r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\indi_img'
group_images_dir = r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\grp_img'

In [36]:
# Load and encode student images
def load_and_encode_images(directory):
    known_encodings = []
    student_names = []
    
    for filename in os.listdir(directory):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Load image
            image_path = os.path.join(directory, filename)
            img = face_recognition.load_image_file(image_path)
            
            # Handle cases with no faces or multiple faces in the image
            try:
                img_encoding = face_recognition.face_encodings(img)[0]
                known_encodings.append(img_encoding)
                student_names.append(os.path.splitext(filename)[0])  # Use the filename without extension as the name
            except IndexError:
                print(f"Warning: No face found in {filename}")
    
    return known_encodings, student_names

In [37]:
# Recognize faces in group photo
def recognize_faces_in_group_photo(image_path, known_encodings, student_names):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image {image_path}")
        return [], []
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    face_locations = face_recognition.face_locations(img_rgb)
    face_encodings = face_recognition.face_encodings(img_rgb, face_locations)
    
    print(f"Detected {len(face_locations)} faces in {image_path}")

    recognized_names = []
    
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = None
        
        if len(face_distances) > 0:
            best_match_index = face_distances.argmin()
        
        if best_match_index is not None and matches[best_match_index]:
            name = student_names[best_match_index]
            recognized_names.append(name)
    
    return recognized_names, face_locations

In [38]:
# Mark attendance in an Excel file
def mark_attendance(names, output_file=r'C:\Users\NIKHAR BEHERA\Anaconda\Face_advr\attendance.xlsx'):
    # Load the existing attendance file or create a new one
    try:
        df = pd.read_excel(output_file)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['Name', 'Time'])

    # Get the current timestamp
    now = datetime.now()
    dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

    # Create a DataFrame for the new attendance entries
    new_entries = pd.DataFrame({
        'Name': [name for name in names if name not in df['Name'].values],
        'Time': [dt_string] * len([name for name in names if name not in df['Name'].values])
    })

    # Concatenate the new entries with the existing DataFrame
    df = pd.concat([df, new_entries], ignore_index=True)

    # Save the updated DataFrame to Excel
    df.to_excel(output_file, index=False)

In [39]:
# Draw bounding boxes on recognized faces
def draw_bounding_boxes(img, face_locations, recognized_names):
    for (top, right, bottom, left), name in zip(face_locations, recognized_names):
        cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [40]:
# Process group photos and mark attendance
def process_group_photos():
    known_encodings, student_names = load_and_encode_images(individual_images_dir)
    
    for filename in os.listdir(group_images_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(group_images_dir, filename)
            recognized_names, face_locations = recognize_faces_in_group_photo(image_path, known_encodings, student_names)
            
            print(f"Recognized in {filename}: {recognized_names}")
            
            mark_attendance(recognized_names)
            
            img = cv2.imread(image_path)
            draw_bounding_boxes(img, face_locations, recognized_names)
            
            cv2.imshow('Attendance', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [2]:
# Run the attendance marking process
if __name__ == "__main__":
    process_group_photos()

NameError: name 'process_group_photos' is not defined